In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-21 06:25:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-10-21 06:25:16 (10.5 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
#load software reviews data from s3.amazonaws.com
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"

spark.sparkContext.addFile(url)

software_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), inferSchema=True, sep="\t")

software_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [5]:
#drop duplicates and incomplete rows
print(software_df.count())
software_df = software_df.dropna()
print(software_df.count())
software_df = software_df.dropDuplicates()
#show total counts of reviews after dropping duplicates and incomplete rows
print(software_df.count())

341931
341913
341913


In [6]:
software_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
vine_reviews = software_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3UTVKJF44E0R6|          1|           46|         69|   N|
|R16IFUMCFXYVR2|          5|            0|          0|   N|
|R21XYG71U6C9IM|          5|            0|          0|   N|
| RSF2WJUKZ7GP4|          5|            0|          0|   N|
| R4OXOE6JIRQJN|          5|            0|          0|   N|
|R2L40N1AZIRKHS|          4|            0|          0|   N|
|R2J99PBZBMC3NQ|          5|            0|          0|   N|
|R11FLCMCYFGDHO|          5|            1|          1|   N|
| REAWQNKQB0DV9|          3|            0|          0|   N|
|R37TVDQAX8RVZ5|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [8]:
#select vine reviews
vine_reviews_yes = vine_reviews.filter(vine_reviews["vine"]=="Y")
#show the total count of vine reviews
print(vine_reviews_yes.count())
vine_reviews_yes.show(10)

10415
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R31Y4SHLQD7UU0|          1|            7|         13|   Y|
| RDMY8U7G00B1D|          4|            2|          2|   Y|
|R12HML0FDHA2UV|          4|            0|          1|   Y|
|R32BYGORDU46YZ|          4|            0|          0|   Y|
| RJZ0MGZNQ9T0E|          4|            0|          0|   Y|
| R8UVR890KYRD7|          4|            0|          0|   Y|
|R367KYVZ1LD8NP|          4|            0|          0|   Y|
|R1ASP9RN7SD3KI|          4|            0|          0|   Y|
|R1JEEKNYAZQ5RI|          4|            0|          1|   Y|
|R3AJ7TCEHRR26O|          4|            0|          1|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [9]:
# mode="append"
# jdbc_url = ""
# config = {"user":,
#           "password":,
#           "driver":"org.postgresql.Driver"}

In [10]:
vine_reviews_yes.write.jdbc(url=jdbc_url, table='software_vine_reviews', mode=mode, properties=config)

Py4JJavaError: ignored